In [1]:
import os
import random
import shutil

# === CONFIGURATION ===
BASE_DIR = "."  # current directory (HMMMMMMM)
SOURCE_IMAGE_DIR = os.path.join(BASE_DIR, "dataset", "images")
SOURCE_LABEL_DIR = os.path.join(BASE_DIR, "dataset", "labels")
OUTPUT_IMAGE_TRAIN = os.path.join(BASE_DIR, "images", "train")
OUTPUT_IMAGE_VAL = os.path.join(BASE_DIR, "images", "val")
OUTPUT_LABEL_TRAIN = os.path.join(BASE_DIR, "labels", "train")
OUTPUT_LABEL_VAL = os.path.join(BASE_DIR, "labels", "val")
SPLIT_RATIO = 0.8  # 80% training, 20% validation

# === MAKE OUTPUT FOLDERS ===
os.makedirs(OUTPUT_IMAGE_TRAIN, exist_ok=True)
os.makedirs(OUTPUT_IMAGE_VAL, exist_ok=True)
os.makedirs(OUTPUT_LABEL_TRAIN, exist_ok=True)
os.makedirs(OUTPUT_LABEL_VAL, exist_ok=True)

# === GET ALL IMAGE FILES ===
all_images = [f for f in os.listdir(SOURCE_IMAGE_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
random.shuffle(all_images)

split_idx = int(len(all_images) * SPLIT_RATIO)
train_images = all_images[:split_idx]
val_images = all_images[split_idx:]

# === COPY FILES TO NEW STRUCTURE ===
def move_files(image_list, image_target_dir, label_target_dir):
    for img_file in image_list:
        img_path = os.path.join(SOURCE_IMAGE_DIR, img_file)
        label_file = os.path.splitext(img_file)[0] + ".txt"
        label_path = os.path.join(SOURCE_LABEL_DIR, label_file)

        if os.path.exists(img_path) and os.path.exists(label_path):
            shutil.copy(img_path, os.path.join(image_target_dir, img_file))
            shutil.copy(label_path, os.path.join(label_target_dir, label_file))
        else:
            print(f"⚠️ Missing pair for {img_file}, skipping.")

move_files(train_images, OUTPUT_IMAGE_TRAIN, OUTPUT_LABEL_TRAIN)
move_files(val_images, OUTPUT_IMAGE_VAL, OUTPUT_LABEL_VAL)

# === CREATE mosquito.yaml ===
yaml_content = f"""path: {os.path.abspath(BASE_DIR).replace(os.sep, '/')}
train: images/train
val: images/val

nc: 2
names: ['mosquito', 'other']
"""

with open(os.path.join(BASE_DIR, "mosquito.yaml"), "w") as f:
    f.write(yaml_content)

print("✅ Dataset split complete and mosquito.yaml created!")


✅ Dataset split complete and mosquito.yaml created!


In [2]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")  # Use yolov8n, yolov8s, etc.

# Train the model
model.train(
    data="mosquito.yaml",
    epochs=30,
    imgsz=640
)


100%|██████████| 6.25M/6.25M [00:00<00:00, 7.11MB/s]


New https://pypi.org/project/ultralytics/8.3.172 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.170  Python-3.12.2 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=mosquito.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=Fals

100%|██████████| 5.35M/5.35M [00:00<00:00, 6.63MB/s]


AMP: checks passed 
train: Fast image access  (ping: 0.00.0 ms, read: 7.53.3 MB/s, size: 33.2 KB)


train: Scanning F:\New folder\labels\train... 766 images, 3 backgrounds, 0 corrupt: 100%|██████████| 766/766 [00:00<00:00, 1298.30it/s]

...

                   all        192        201      0.923      0.914      0.942      0.586



30 epochs completed in 0.038 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.170  Python-3.12.2 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  5.97it/s]


                   all        192        201      0.931      0.921      0.941      0.586
              mosquito        140        146      0.973      0.973      0.992      0.467
                 other         52         55      0.889      0.869      0.891      0.705
Speed: 0.2ms preprocess, 0.5ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000013B124D8470>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [3]:
from ultralytics import YOLO

# Load your trained model
model = YOLO("runs/detect/train/weights/best.pt")  # or wherever your best.pt is saved

# Run evaluation
metrics = model.val()  # uses the same data=mosquito.yaml by default if it was used during training


Ultralytics 8.3.170  Python-3.12.2 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1016.21223.1 MB/s, size: 133.8 KB)


val: Scanning F:\New folder\labels\val.cache... 192 images, 0 backgrounds, 0 corrupt: 100%|██████████| 192/192 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:03<00:00,  3.33it/s]


                   all        192        201      0.932      0.924      0.942      0.591
              mosquito        140        146      0.975      0.979      0.994      0.478
                 other         52         55      0.889      0.869       0.89      0.704
Speed: 0.6ms preprocess, 2.0ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\val


In [4]:
results = model.predict(source="images/val", conf=0.25, save=True)


image 1/192 f:\New folder\images\val\image_0004.jpg: 640x640 1 other, 8.6ms
image 2/192 f:\New folder\images\val\image_0008.jpg: 640x640 1 other, 10.9ms
image 3/192 f:\New folder\images\val\image_0010.jpg: 640x640 4 others, 12.8ms
image 4/192 f:\New folder\images\val\image_0012.jpg: 640x640 1 other, 14.3ms
image 5/192 f:\New folder\images\val\image_0014.jpg: 640x640 1 other, 12.0ms
image 6/192 f:\New folder\images\val\image_0018.jpg: 640x640 1 other, 5.4ms
image 7/192 f:\New folder\images\val\image_0032.jpg: 640x640 1 other, 4.3ms
image 8/192 f:\New folder\images\val\image_0038.jpg: 640x640 1 other, 4.3ms
image 9/192 f:\New folder\images\val\image_0040.jpg: 640x640 2 others, 4.2ms
image 10/192 f:\New folder\images\val\image_0041.jpg: 640x640 1 other, 4.6ms
image 11/192 f:\New folder\images\val\image_0044.jpg: 640x640 1 other, 5.3ms
image 12/192 f:\New folder\images\val\image_0049.jpg: 640x640 1 other, 3.5ms
image 13/192 f:\New folder\images\val\image_0055.jpg: 640x640 1 other, 3.7ms
i

In [1]:
from ultralytics import YOLO
import cv2

# Load your trained model
model = YOLO("runs/detect/train/weights/best.pt")  # Replace with actual model path

# Start webcam
cap = cv2.VideoCapture(0)

# Check if webcam is opened
if not cap.isOpened():
    print("Error: Cannot open webcam")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run detection
    results = model(frame)

    # Access detection result
    detections = results[0]
    
    # Filter detections by class (if your model has multiple classes)
    # For single-class mosquito model, count all boxes
    num_mosquitoes = len(detections.boxes)

    # Annotate frame with bounding boxes
    annotated_frame = detections.plot()

    # Add mosquito count text to the frame
    cv2.putText(annotated_frame, f"Mosquitoes: {num_mosquitoes}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show the frame
    cv2.imshow("Mosquito Detection", annotated_frame)

    # Break loop with 'q'
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 60.2ms
Speed: 6.8ms preprocess, 60.2ms inference, 47.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 4.8ms
Speed: 1.8ms preprocess, 4.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 4.3ms
Speed: 1.0ms preprocess, 4.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.4ms
Speed: 1.0ms preprocess, 6.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 3.9ms
Speed: 1.0ms preprocess, 3.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.6ms
Speed: 1.2ms preprocess, 6.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 4.0ms
Speed: 1.2ms preprocess, 4.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 5.5ms
Speed: 1.1ms preprocess, 5.5ms inference, 0.6

: 